# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [2]:
from google.colab import userdata
userdata.get('HF_TOKEN')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Cloning https://github.com/huggingface/accelerate.git to /private/var/folders/1k/zl09cwlx1dn_f7gpczg8q5f00000gp/T/pip-req-build-lnvtu7u6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /private/var/folders/1k/zl09cwlx1dn_f7gpczg8q5f00000gp/T/pip-req-build-lnvtu7u6
  Resolved https://github.com/huggingface/accelerate.git to commit 445ef7138648ee6a9c51375aab82ff9600c4d221
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.13.0.dev0-py3-none-any.whl size=382176 sha256=9fab7f1f98820b88568f2aee94b49f83945293647e1bd1d84b8699c4a26a67e6
  Stored in directory: /private/var/folders/1k/zl09cwlx1dn_f7gpczg8q5f00000gp/T/pip-ephem-wheel-cache-c8vzxy81/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35f8a8365
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installati

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate
from mlx_lm import load, generate

In [4]:
model_name = "mlx-community/defog-sqlcoder-7b-2"
model, tokenizer = load(model_name)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [ ]:
# bnb_config = BitsAndBytesConfig(
#   load_in_4bit=True,
#   bnb_4bit_use_double_quant=True,
#   bnb_4bit_quant_type="nf4",
#   bnb_4bit_compute_dtype=torch.bfloat16
# )


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [ ]:
# foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
#                     quantization_config=bnb_config,
#                     device_map='auto',
#                     use_cache = True)

OSError: mlx-community/defog-sqlcoder-7b-2 does not appear to have a file named pytorch_model.bin or model.safetensors.

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

This function wraps the call to *model.generate*

In [6]:
# This function wraps the MLX generation
def get_outputs(model, prompt, max_new_tokens=400):
    # MLX handles tokenization and detokenization automatically in the generate() function
    response = generate(
        model,
        tokenizer,
        prompt=prompt,
        verbose=True, # Set to True to see the text generating in real-time (Matrix style)
        max_tokens=max_new_tokens
    )
    return response

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [ ]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE hospital_cases, cost, sex_practique, drugs.

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [9]:
sp_nl2sql = sp_nl2sql.format(question="What is the best drug in relation between the most cheapest, less hospital cases and cost") # Or whatever question is in the lab
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `What is the best drug in relation between the most cheapest, less hospital cases and cost`:
    ```sql3
    


In [10]:
# Execute directly on your M4
print("--- GENERATING SQL ---")
sql_result = get_outputs(model, sp_nl2sql, max_new_tokens=400)

# The result is already a string, no need to decode
print("\n--- FINAL OUTPUT ---")
print(sql_result)

--- GENERATING SQL ---
SELECT d.name, d.price, COUNT(h.id) AS hospital_cases FROM drugs d JOIN hospitals h ON d.id = h.drug_id GROUP BY d.name, d.price ORDER BY d.price ASC LIMIT 1;
Prompt: 157 tokens, 101.145 tokens-per-sec
Generation: 59 tokens, 26.681 tokens-per-sec
Peak memory: 13.686 GB

--- FINAL OUTPUT ---
SELECT d.name, d.price, COUNT(h.id) AS hospital_cases FROM drugs d JOIN hospitals h ON d.id = h.drug_id GROUP BY d.name, d.price ORDER BY d.price ASC LIMIT 1;


In [ ]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [12]:
# 1. Generamos el SQL usando tu M4 (Esto tarda unos segundos)
print("🧠 Pensando SQL...")
response = get_outputs(model, sp_nl2sql)

# 2. Limpieza de texto (Post-processing)
# El modelo a veces envuelve el código en ```sql ... ```
cleaned_sql = response
if "```" in response:
    # Si hay bloques de código, nos quedamos con el contenido
    # Ajustamos para que coja tanto 'sql' como 'sql3' o genérico
    cleaned_sql = response.split("```")
    # Buscamos el trozo más largo que parezca una query o cogemos el segundo bloque
    if len(cleaned_sql) > 1:
        cleaned_sql = cleaned_sql[1]
        # Quitamos la palabra 'sql' o 'sql3' si se ha colado al principio
        if cleaned_sql.startswith("sql"):
            cleaned_sql = cleaned_sql[3:]
        elif cleaned_sql.startswith("3"): # Por si era sql3
            cleaned_sql = cleaned_sql[1:]

# 3. Imprimimos el resultado final limpio
print("\n--- SQL GENERADO ---")
print(cleaned_sql.strip() + ";")

🧠 Pensando SQL...
SELECT d.name, d.price, COUNT(h.id) AS hospital_cases FROM drugs d JOIN hospitals h ON d.id = h.drug_id GROUP BY d.name, d.price ORDER BY d.price ASC LIMIT 1;
Prompt: 157 tokens, 333.316 tokens-per-sec
Generation: 59 tokens, 26.665 tokens-per-sec
Peak memory: 13.686 GB

--- SQL GENERADO ---
SELECT d.name, d.price, COUNT(h.id) AS hospital_cases FROM drugs d JOIN hospitals h ON d.id = h.drug_id GROUP BY d.name, d.price ORDER BY d.price ASC LIMIT 1;;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [15]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   hospital_cases, cost, sex_practique, drugs.

    ### Response
    Question: How many female patients were treated last month?
    SQL: SELECT COUNT(*) FROM hospital_cases WHERE sex_practique = 'F' AND date > '2026-01-01';

    Question: What is the total cost for the drug 'Aspirin'?
    SQL: SELECT SUM(cost) FROM drugs WHERE drug_name = 'Aspirin';

    Question: List the top 5 most expensive hospital cases.
    SQL: SELECT * FROM hospital_cases ORDER BY total_cost DESC LIMIT 5;
    `{question}`:
    ```sql3
    """


In [16]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   hospital_cases, cost, sex_practique, drugs.

    ### Response
    Question: How many female patients were treated last month?
    SQL: SELECT COUNT(*) FROM hospital_cases WHERE sex_practique = 'F' AND date > '2026-01-01';

    Question: What is the total cost for the drug 'Aspirin'?
    SQL: SELECT SUM(cost) FROM drugs WHERE drug_name = 'Aspirin';

    Question: List the top 5 most expensive hospital cases.
    SQL: SELECT * FROM hospital_cases ORDER BY total_cost DESC LIMIT 5;
    `Return The name of

In [18]:
# No tokenizer, no .to('cuda'), no decoding needed. 
# MLX handles the string directly.
SQL = get_outputs(model, sp_nl2sql2)

print("\n--- GENERATED SQL ---")
print(SQL)

SELECT employee_name FROM employees ORDER BY total_cost DESC LIMIT 1;
Prompt: 287 tokens, 219.444 tokens-per-sec
Generation: 17 tokens, 27.768 tokens-per-sec
Peak memory: 13.881 GB

--- GENERATED SQL ---
SELECT employee_name FROM employees ORDER BY total_cost DESC LIMIT 1;


In [19]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

S;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [21]:
sp_nl2sql3b = """
    ### Instructions:
    Your task is convert a question into a SQL query, given a SQL database schema.
    Adhere to these rules:
    - **Deliberately go through the question and database schema word by word** to appropriately answer the question
    - **Use the samples SQL In the ### Samples section to learn more about the Databases structure

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE hospital_cases (case_id INT, patient_id INT, admission_date DATE, total_cost FLOAT, sex_practique TEXT);
    CREATE TABLE drugs (drug_id INT, case_id INT, drug_name TEXT, cost FLOAT);
    
    ### Samples
    Question: Total cost for 'M' patients?
    SQL: SELECT SUM(total_cost) FROM hospital_cases WHERE sex_practique = 'M';

    Question: Drugs for case 101?
    SQL: SELECT drug_name FROM drugs WHERE case_id = 101;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql
    """

In [23]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best drug paid employee that like sex so much")
print (sp_nl2sql3)


    ### Instructions:
    Your task is convert a question into a SQL query, given a SQL database schema.
    Adhere to these rules:
    - **Deliberately go through the question and database schema word by word** to appropriately answer the question
    - **Use the samples SQL In the ### Samples section to learn more about the Databases structure

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE hospital_cases (case_id INT, patient_id INT, admission_date DATE, total_cost FLOAT, sex_practique TEXT);
    CREATE TABLE drugs (drug_id INT, case_id INT, drug_name TEXT, cost FLOAT);
    
    ### Samples
    Question: Total cost for 'M' patients?
    SQL: SELECT SUM(total_cost) FROM hospital_cases WHERE sex_practique = 'M';

    Question: Drugs for case 101?
    SQL: SELECT drug_name FROM drugs WHERE case_id = 101;

    ### Response
    Based on your instructions, here is 

In [24]:
# No tokenizer, no .to('cuda'), no decoding needed. 
# MLX handles the string directly.
SQL = get_outputs(model, sp_nl2sql2)

print("\n--- GENERATED SQL ---")
print(SQL)

SELECT employee_name FROM employees ORDER BY total_cost DESC LIMIT 1;
Prompt: 287 tokens, 475.148 tokens-per-sec
Generation: 17 tokens, 27.695 tokens-per-sec
Peak memory: 13.881 GB

--- GENERATED SQL ---
SELECT employee_name FROM employees ORDER BY total_cost DESC LIMIT 1;


In [25]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

S;


#Now the question in spanish.


In [27]:
sp_nl2sql3 = sp_nl2sql3b.format(question="dIME EL TRABAJADOR QUE COBRA MAS")
print (sp_nl2sql3)


    ### Instructions:
    Your task is convert a question into a SQL query, given a SQL database schema.
    Adhere to these rules:
    - **Deliberately go through the question and database schema word by word** to appropriately answer the question
    - **Use the samples SQL In the ### Samples section to learn more about the Databases structure

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE hospital_cases (case_id INT, patient_id INT, admission_date DATE, total_cost FLOAT, sex_practique TEXT);
    CREATE TABLE drugs (drug_id INT, case_id INT, drug_name TEXT, cost FLOAT);
    
    ### Samples
    Question: Total cost for 'M' patients?
    SQL: SELECT SUM(total_cost) FROM hospital_cases WHERE sex_practique = 'M';

    Question: Drugs for case 101?
    SQL: SELECT drug_name FROM drugs WHERE case_id = 101;

    ### Response
    Based on your instructions, here is 

In [28]:
# No tokenizer, no .to('cuda'), no decoding needed. 
# MLX handles the string directly.
SQL = get_outputs(model, sp_nl2sql2)

print("\n--- GENERATED SQL ---")
print(SQL)

SELECT employee_name FROM employees ORDER BY total_cost DESC LIMIT 1;
Prompt: 287 tokens, 474.303 tokens-per-sec
Generation: 17 tokens, 27.475 tokens-per-sec
Peak memory: 13.881 GB

--- GENERATED SQL ---
SELECT employee_name FROM employees ORDER BY total_cost DESC LIMIT 1;


In [29]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

S;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?